# <Section 01 Project: 다음 분기에 어떤 게임을 설계해야 할까?>


In [ ]:
#다음 분기에 출고량이 높을 것이라 예상되는 게임

## 1. EDA

In [ ]:
import pandas as pd

df = pd.read_csv("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv")
df.drop(["Unnamed: 0"], axis = 1, inplace = True)
df.head()

In [ ]:
df.info()

In [ ]:
import numpy as np

def int_to_year(num):
    this_year = 21 #2021년도
    if num < 0 or num == np.nan:
        return num.replace(num, np.nan)
    elif num < this_year:
        return int(num) + 2000
    elif num < 100:
        return int(num) + 1900
    else:
        return int(num)

def convert_str_to_number(string):
    if "K" in string:
        return float(string.replace("K", "")) * (10**3)
    elif "M" in string:
        return float(string.replace("M", "")) * (10**6)
    else:
        return float(string) * (10**6)

df.dropna(axis = 0, inplace = True) #결측치 제거

df["Year"] = df["Year"].apply(int_to_year)
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    df[Sales] = df[Sales].apply(convert_str_to_number)

df.reset_index(drop = True, inplace = True)

#확인
df.info()
print()
df

## 2. 분석

### 1. "**지역**"과 "**선호하는 게임장르**"는 연관이 있을것인가?
= 선호하는 게임장르는 곧 출고량과 직결되므로 위 물음을 답하기 위해 Chi-square test을 사용할 수 있음.

    (1) 귀무가설: 지역과 게임장르는 출고량에 대해서 서로 연관이 있을 것이다.
    (2) 대안가설: 지역과 게임장르는 출고량과 대해서 서로 연관이 없을 것이다.
    (3) 신뢰도: 95 %

In [ ]:
df_Genre_sales = df.drop(["Name", "Platform", "Year", "Publisher"], axis = 1, inplace = False)
df_Genre_sales

In [ ]:
Genre_Region_sum_list = []
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    Genre_sum = df_Genre_sales[Sales].groupby(df_Genre_sales["Genre"]).sum()
    Genre_Region_sum_list.append(Genre_sum)

df_Genre_Region_sum = pd.concat(Genre_Region_sum_list, axis = 1)
df_Genre_Region_sum

In [ ]:
df_Genre_Region_sum.corrwith(other = df_Genre_Region_sum[["NA_Sales"]], method='pearson')

In [ ]:
from scipy import stats

chi1 = stats.chi2_contingency(df_Genre_Region_sum, correction = False)
pv1 = chi1[1]
pv1

In [ ]:
#p-value가 0.05 이하이므로 귀무가설을 기각, 대안가설을 채택함.
#즉, 지역과 게임 장르는 출고량에 대해서 서로 연관이 없다고 볼 수 있다.

1. 연도별로 게임 장르의 트렌드를 통해 다음 분기에 설계할 게임의 장르를 추릴 수 있을 것이다.

    (1) 연도별 - 지역별로 출고량의 합 구하기 및 비율 구하기
        연도별로 특정 지역의 전체 출고량(그 지역의 게임시장의 크기)를 알 수 있음.

    (2) 연도별 - 장르별로 출고량의 합 구하기
        연도별로 어떤 장르의 출고량이 가장 높은지 알 수 있음.

    (3) 장르별 - 지역별로 출고량의 합 구하기
        





In [ ]:
df_year_sales = df.drop(["Name", "Platform", "Genre", "Publisher"], axis = 1, inplace = False)

In [ ]:
Year_Region_sum_list = []
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    Year_sum = df_year_sales[Sales].groupby(df_year_sales["Year"]).sum()
    Year_Region_sum_list.append(Year_sum)

df_Year_Region_sum = pd.concat(Year_Region_sum_list, axis = 1)
df_Year_Region_sum["All_Sales"] = df_Year_Region_sum.sum(axis = 1)

df_Year_Region_All_Sales = df_Year_Region_sum["All_Sales"]

df_Year_Region_sum.head()

In [ ]:
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    df_Year_Region_sum[Sales] = df_Year_Region_sum[Sales] / df_Year_Region_sum["All_Sales"]

df_Year_Region_sum.drop(["All_Sales"], axis = 1, inplace = True)
df_Year_Region_sum.drop([2017, 2020], axis = 0, inplace = True) #데이터 수 적어 신뢰성 부족하다고 판단.
df_Year_Region_sum = df_Year_Region_sum * 100
df_Year_Region_sum.head()

In [ ]:
import matplotlib.pyplot as plt

df_Year_Region_sum.plot(kind = "bar", stacked = True, fontsize = 9, figsize = (13, 8))
plt.xticks(rotation = 40)
plt.axhline(y = 70, color = "r", linewidth = 1)

plt.xlabel("Year")
plt.ylabel("Market Share (%)")
plt.title("< Market share by continent (country) >", fontsize = 12)

plt.show()

In [ ]:
#위 그래프는 게임의 장르에 상관없이 연도별로 각 지역별 출고량을 누적 막대 그래프로 나타낸 자료 입니다.
# >> 북미 + 유럽지역의 출고량 압도적임.

In [ ]:
#연도별 게임장르의 트렌드 분석

In [ ]:
df["All_Sales"] = df["NA_Sales"] + df["EU_Sales"] + df["JP_Sales"] + df["Other_Sales"]
df_Year_Genre = df.drop(["Name", "Platform", "Publisher", "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"], axis = 1, inplace = False)
df_Year_Genre.head()

In [ ]:
Year_Genre_sum = df_Year_Genre.groupby(["Year", "Genre"])["All_Sales"].sum()
Year_Genre_ratio = Year_Genre_sum / df_Year_Region_All_Sales
df_Year_Genre_ratio = pd.DataFrame(Year_Genre_ratio)
df_Year_Genre_ratio.drop([2017, 2020], axis = 0, inplace = True) #데이터 수 적어 신뢰성 부족하다고 판단.
df_Year_Genre_ratio = df_Year_Genre_ratio * 100 #백분율

In [ ]:
colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan", "tan", "slateblue"]

df_Year_Genre_ratio.unstack().plot(kind = 'line',
                                   fontsize = 8,
                                   figsize = (7, 13),
                                   color = colors,
                                   subplots = True,
                                   ylim = [0, 40],
                                   grid = True,
                                   xlabel = "Year",
                                   ylabel = "Share(%)")

plt.show()

In [ ]:
df_Year_Genre_sum = pd.DataFrame(Year_Genre_sum)

Year_Genre_count = df_Year_Genre.groupby(["Year", "Genre"])["Genre"].count()
df_Year_Genre_count = pd.DataFrame(Year_Genre_count)

df_Year_Genre_sum["Count"] = df_Year_Genre_count["Genre"]

df_Year_Genre_sum.reset_index(level = 0, drop = True, inplace = True)

#확인
df_Year_Genre_sum

In [ ]:
from sklearn.preprocessing import StandardScaler
import seaborn as sns

labels = df_Year_Genre_sum.index

standard_df_Year_Genre = StandardScaler().fit_transform(df_Year_Genre_sum)
df_standard_Year_Genre = pd.DataFrame(standard_df_Year_Genre, columns = ["All_Sales", "Count"])

df_standard_Year_Genre["Labels"] = labels

#시각화
sns.lmplot(x = 'Count',
           y = 'All_Sales',
           hue = 'Labels',
           #col = 'Labels',
           line_kws={'label':"tip"},
           data = df_standard_Year_Genre,
           #col_wrap = 3,
           order = 1)

In [ ]:
#나라별로 보다 선호하는 장르
#나라별로 보다 선호하는 플랫폼
#위에 나온 그래프는 전체 지역에서 선호하는 장르를 추린것임.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

df_Total_Sales = df[["All_Sales"]]
scaler = StandardScaler()
df_Total_Sales_standard = scaler.fit_transform(df_Total_Sales)

sum_of_squared_distances = []
K = range(1, 15)
for k in K:
    km = KMeans(n_clusters = k)
    km = km.fit(df_Total_Sales_standard)
    sum_of_squared_distances.append(km.inertia_)

plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()


In [ ]:
scaler = StandardScaler()
All_Sales_std = scaler.fit_transform(df[["All_Sales"]])
df_All_Sales_std = pd.DataFrame(All_Sales_std, columns = ["std_All_Sales"])

kmeans = KMeans(n_clusters = 6, random_state = 7)
kmeans.fit(df_All_Sales_std)
K_labels = kmeans.labels_
df_cluster = pd.DataFrame(K_labels, columns = ["Sale_rating"])

df["std_All_Sales"] = df_All_Sales_std["std_All_Sales"]
df["Sale_rating"] = df_cluster["Sale_rating"]
df.head()

In [ ]:
#scatterplot 그리기
sns.scatterplot(x = "Year", y = "std_All_Sales", hue = "Sale_rating", data = df)

In [ ]:
df_Sales_high = df.query("Sale_rating > 0") #상위 14.57 %
df_Sales_low = df.query('Sale_rating == 0') #상위 85.43 $
df_Sales_highest = df.query('(Sale_rating == 1) or (Sale_rating == 5) or (Sale_rating == 3)') #상위 0.5 %

#높은 출고율이 전체 출고율에 차지하는 비율
rate = (df_Sales_highest.shape[0] / (df_Sales_high.shape[0] + df_Sales_low.shape[0])) * 100

print("'높은 출고량'에 대한 기준을 상위", round(high_rate, 2), "% 에 대한 출고량으로 두었습니다. \n")

#EDA
df_Sales_high.reset_index(drop = True, inplace = True)
df_Sales_high = df_Sales_high.drop(["Sale_rating", "std_All_Sales", "Name", "Publisher"], axis = 1, inplace = False)
df_Sales_high

In [ ]:
# 장르랑 플랫폼이랑 묶어서 연관성 있는지 가설검정 >>
df_high_Plat_Genre_count = pd.crosstab([df_Sales_high.Year, df_Sales_high.Platform], df_Sales_high.Genre,  margins=True)
df_Plat_Year_count = pd.crosstab(df.Platform, df.Year, margins = True)
#df_highest_Plat_Genre_count = pd.crosstab(df.Year, df_Sales_highest.Platform, df_Sales_highest.Genre, margins=True)


#df_high_Plat_Genre_count
# 플랫폼 별로 어떤 지역
# 2600단종: https://ko.wikipedia.org/wiki/%EC%95%84%ED%83%80%EB%A6%AC_2600#%ED%9A%8C%EC%83%9D
df_Plat_Year_count = df_Plat_Year_count.drop([2017, 2020, "All"], axis = 1, inplace = False)
df_Plat_Year_count = df_Plat_Year_count.drop(df_Plat_Year_count.columns[:20], axis = 1, inplace = False)
df_Plat_Year_count = df_Plat_Year_count.drop(["All"], axis = 0, inplace = False)

import seaborn as sns
plt.rcParams["figure.figsize"] = (30,10)
sns.heatmap(df_Plat_Year_count * 0.01,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 3 , #컬러차트 영역 -1 ~ +1
          )

In [ ]:
from sklearn.decomposition import PCA

#정보 손실이 적을(분산이 큰) feature의 고유벡터, 고유값 찾기
pca = PCA(n_components = 2) #주성분으로 할 feature의 수 설정
pca.fit(df_Sales_standard) #분산-공분산 Matrix의 고유값, 고유벡터 반환
ratio = pca.explained_variance_ratio_ #array 값의 합: 원본 데이터의 반영(설명) %

#2개의 Feature에 대한 데이터를 고유벡터에 Projection
#pg_2_projection = pca.fit_transform(df_Total_Sales_standard)
print("지역별 출고량에 대해 2개의 Feature로 PCA를 했을 때 원본 데이터를 반영하는 비율: ", round(ratio.sum() * 100, 2), "%" )

pro = pca.fit_transform(df_Sales_standard)

In [ ]:
from sklearn.preprocessing import StandardScaler

Sales_header = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]
Sales_standard = scaler.fit_transform(df.iloc[:, 5:9])
df_Sales_standard = pd.DataFrame(Sales_standard, columns = Sales_header)
df_Sales_standard.head()

In [ ]:
pro = pca.fit_transform(df_Sales_standard)

In [ ]:
df_pro = pd.DataFrame(data = pro, columns = ["PC1", "PC2"])
df_pro.head()

In [ ]:
kmeans = KMeans(n_clusters = 6)
kmeans.fit(pro)
K_labels = kmeans.labels_
df_cluster = pd.DataFrame(K_labels, columns = ["Sale_rating"])
df["Sale_rating"] = df_cluster["Sale_rating"]
df

In [ ]:
df_pro["Labels"] = K_labels

#scatterplot 그리기
sns.scatterplot(x = "PC1", y = "PC2", hue = "Labels", data = df_pro)